In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns

def load_data(csv_file):
    """Carga datos desde CSV y asegura formato correcto de timestamp"""
    df = pd.read_csv(csv_file)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

def create_intervals(df, interval_seconds):
    """
    Crea intervalos de tiempo y agrupa los datos
    Returns: DataFrame con métricas por intervalo
    """
    # Crear índice de intervalos
    start_time = df['timestamp'].min()
    end_time = df['timestamp'].max()
    
    # Redondear al intervalo más cercano
    df['interval'] = ((df['timestamp'] - start_time).dt.total_seconds() 
                     // interval_seconds).astype(int)
    
    # Agrupar por intervalo
    interval_metrics = df.groupby('interval').agg({
        'mac': ['count', 'nunique'],  # count total y MACs únicas
        'timestamp': 'first'  # tiempo inicio del intervalo
    })
    
    interval_metrics.columns = ['total_events', 'unique_macs', 'interval_start']
    return interval_metrics.reset_index()

def plot_interval_analysis(interval_metrics, interval_seconds, plot_type='line'):
    """
    Genera visualizaciones de los datos por intervalo
    """
    plt.style.use('seaborn')
    
    if plot_type == 'line':
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
        
        # Plot 1: MACs únicas
        ax1.plot(interval_metrics['interval'], interval_metrics['unique_macs'], 
                'b-', label='MACs únicas', marker='o')
        ax1.axhline(y=interval_metrics['unique_macs'].mean(), color='r', 
                   linestyle='--', label='Media MACs')
        ax1.axhline(y=interval_metrics['unique_macs'].max(), color='g', 
                   linestyle='--', label='Máximo MACs')
        
        ax1.set_xlabel('Índice de Intervalo')
        ax1.set_ylabel('Número de MACs Únicas')
        ax1.set_title(f'MACs Únicas por Intervalo ({interval_seconds}s)')
        ax1.grid(True)
        ax1.legend()
        
        # Plot 2: Eventos totales
        ax2.plot(interval_metrics['interval'], interval_metrics['total_events'], 
                'purple', label='Total Eventos', marker='s')
        ax2.axhline(y=interval_metrics['total_events'].mean(), color='r', 
                   linestyle='--', label='Media Eventos')
        
        ax2.set_xlabel('Índice de Intervalo')
        ax2.set_ylabel('Número Total de Eventos')
        ax2.set_title(f'Eventos Totales por Intervalo ({interval_seconds}s)')
        ax2.grid(True)
        ax2.legend()
        
    elif plot_type == 'box':
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Box plot para MACs únicas
        sns.boxplot(y=interval_metrics['unique_macs'], ax=ax1)
        ax1.set_title('Distribución de MACs Únicas')
        ax1.set_ylabel('Número de MACs')
        
        # Box plot para eventos totales
        sns.boxplot(y=interval_metrics['total_events'], ax=ax2)
        ax2.set_title('Distribución de Eventos Totales')
        ax2.set_ylabel('Número de Eventos')
    
    plt.tight_layout()
    plt.show()

def analyze_intervals(df, interval_seconds):
    """
    Realiza análisis estadístico de los intervalos
    """
    interval_metrics = create_intervals(df, interval_seconds)
    
    stats = {
        'Intervalo (segundos)': interval_seconds,
        'Número total de intervalos': len(interval_metrics),
        'MACs únicas - Media': interval_metrics['unique_macs'].mean(),
        'MACs únicas - Máximo': interval_metrics['unique_macs'].max(),
        'MACs únicas - Moda': interval_metrics['unique_macs'].mode().iloc[0],
        'Eventos totales - Media': interval_metrics['total_events'].mean(),
        'Eventos totales - Máximo': interval_metrics['total_events'].max(),
    }
    
    print("\nEstadísticas del análisis:")
    print("-" * 50)
    for key, value in stats.items():
        print(f"{key}: {value:.2f}")
        
    return interval_metrics, stats

In [ ]:
 # Cargar datos
    df = load_data("ble_data.csv")
    
    # Analizar con diferentes intervalos
    intervals = [1, 2, 3, 4, 5, 10]  # segundos
    
    for interval in intervals:
        print(f"\nAnálisis para intervalo de {interval} segundos")
        print("=" * 50)
        
        # Realizar análisis
        interval_metrics, stats = analyze_intervals(df, interval)
        
        # Generar visualizaciones
        plot_interval_analysis(interval_metrics, interval, plot_type='line')
        plot_interval_analysis(interval_metrics, interval, plot_type='box')